<a href="https://colab.research.google.com/github/Matchingg/nietzsche-ai/blob/main/Nietzsche_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Dependencies

In [ ]:
import tensorflow as tf
import numpy as np
import os

Import File

In [ ]:
path_to_file = tf.keras.utils.get_file('nietzsche.txt', 'https://s3.amazonaws.com/text-datasets/nietzsche.txt')

614400/600901 [==============================] - 1s 1us/step


Read File

In [ ]:
text = open(path_to_file,'rb').read()
text = text.decode(encoding='utf-8')
text = text.replace('\n', ' ')
print('Total number of characters is:', len(text))
print('The first 100 characters are as follows:\n', text[:100])

Total number of characters is: 600893
The first 100 characters are as follows:
 PREFACE   SUPPOSING that Truth is a woman--what then? Is there not ground for suspecting that all ph


Vectorizing Text

In [ ]:
vocab = sorted(set(text))
print('The number of unique characters is', len(vocab))
print('A slice of the unique characters set:\n', vocab[:10])

The number of unique characters is 83
A slice of the unique characters set:
 [' ', '!', '"', "'", '(', ')', ',', '-', '.', '0']


In [ ]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

Creating the Dataset

In [ ]:
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
seq_length = 100
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text = chunk[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>


Building the Model

In [ ]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
          tf.keras.layers.GRU(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
          tf.keras.layers.Dense(vocab_size)])
  return model

In [ ]:
model = build_model(vocab_size = len(vocab),
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units,
                    batch_size=BATCH_SIZE)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           21248     
                                                                 
 gru (GRU)                   (64, None, 1024)          3938304   
                                                                 
 dense (Dense)               (64, None, 83)            85075     
                                                                 
Total params: 4,044,627
Trainable params: 4,044,627
Non-trainable params: 0
_________________________________________________________________


Compiling and Training

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [ ]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix, save_weights_only=True)

In [ ]:
EPOCHS = 30
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
92/92 [==============================] - 15s 53ms/step - loss: 2.9371
Epoch 2/30
92/92 [==============================] - 6s 53ms/step - loss: 2.2604
Epoch 3/30
92/92 [==============================] - 6s 53ms/step - loss: 1.9962
Epoch 4/30
92/92 [==============================] - 6s 54ms/step - loss: 1.7813
Epoch 5/30
92/92 [==============================] - 6s 54ms/step - loss: 1.6180
Epoch 6/30
92/92 [==============================] - 6s 55ms/step - loss: 1.5042
Epoch 7/30
92/92 [==============================] - 6s 54ms/step - loss: 1.4228
Epoch 8/30
92/92 [==============================] - 6s 55ms/step - loss: 1.3610
Epoch 9/30
92/92 [==============================] - 6s 55ms/step - loss: 1.3127
Epoch 10/30
92/92 [==============================] - 6s 55ms/step - loss: 1.2702
Epoch 11/30
92/92 [==============================] - 6s 57ms/step - loss: 1.2313
Epoch 12/30
92/92 [==============================] - 6s 56ms/step - loss: 1.1950
Epoch 13/30
92/92 [=================

Generating New Text

In [ ]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [ ]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

In [ ]:
import pickle
pickle.dump(model, open('model.pkl', 'wb'))

INFO:tensorflow:Assets written to: ram://40586371-d99f-4ad8-814e-ea67b9a76532/assets


INFO:tensorflow:Assets written to: ram://40586371-d99f-4ad8-814e-ea67b9a76532/assets


# New section

In [ ]:
def generate_text(model, num_generate, temperature, start_string):
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    input_eval = tf.expand_dims([predicted_id], 0)
    text_generated.append(idx2char[predicted_id])

  generated_text = start_string + ''.join(text_generated)
  generated_text = generated_text[:generated_text.rfind(' ')] + '.'
  generated_text = ''.join(i for i in generated_text if not i.isdigit())
  return generated_text

In [ ]:
generated_text = generate_text(model, num_generate=500, temperature=1, start_string='Plato')
print(generated_text)

Platon of "Trificing, and melianity generally estimate the thing is shy no means eternal, for their protection and passions within associated with palidn tially suffers from nature, for the end, however, though he accepts, not as he present may revenges it and its ultimate conception hitherto. There is not our duties as clear as the unselfish man: niner, and the French Revolution (in the case of SPPICITACE for himself; the same, so that even Greek life as frle of worship results universally bare.
